In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from itertools import product
from tqdm.notebook import tqdm
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np
import os
import ast

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

In [3]:
df = pd.read_csv('./datasets/data_treino_dv_df_2000_2010.csv')
df.head(1)

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7


In [4]:
df.columns = ['HORA','WIND_DIR_HOR','WIND_VEL_HOR','HUM_REL_MAX_ANT','HUM_REL_MIN_ANT','TEMP_MAX_ANT','TEMP_MIN_ANT','HUM_REL_HOR','PRES_ATM_NIV','PREC_HOR','RAFAGA_VIENTO','PRES_ATM_MAX_ANT','PRES_ATM_MIN_ANT']
df.head(2)

,HORA,WIND_DIR_HOR,WIND_VEL_HOR,HUM_REL_MAX_ANT,HUM_REL_MIN_ANT,TEMP_MAX_ANT,TEMP_MIN_ANT,HUM_REL_HOR,PRES_ATM_NIV,PREC_HOR,RAFAGA_VIENTO,PRES_ATM_MAX_ANT,PRES_ATM_MIN_ANT
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2


In [5]:
df.drop(columns='HORA', inplace= True)

In [6]:
def create_shifted_lagged_features(
    df: pd.DataFrame,
    response_col: str,
    num_lags: int,
    response_shift: int = 0,
    output_path: str = None
) -> pd.DataFrame:
    """
    Genera un DataFrame con características de retardo (lag features) para modelado,
    y opcionalmente lo exporta a un archivo Excel, permitiendo desplazar la variable de respuesta.

    Parámetros:
    ----------
    df : pd.DataFrame
        DataFrame original que contiene las columnas de respuesta y predictoras.
        El índice debe estar ordenado temporalmente.

    response_col : str
        Nombre de la columna que se utilizará como variable de respuesta.

    num_lags : int
        Número de períodos de retardo a considerar para las características.

    response_shift : int, opcional (por defecto 0)
        Número de períodos para desplazar la variable de respuesta.
        - Si es positivo, desplaza la respuesta hacia el futuro (predice el futuro).
        - Si es negativo, desplaza la respuesta hacia el pasado.
        - Si es 0, no se desplaza la respuesta.

    output_path : str, opcional (por defecto None)
        Ruta completa donde se exportará el DataFrame resultante en formato Excel.
        Si es None, no se exportará el archivo.

    Retorna:
    -------
    pd.DataFrame
        DataFrame con las características de retardo y la variable de respuesta.
        Las filas con valores faltantes serán eliminadas.
    """

    if response_col not in df.columns:
        raise ValueError(f"La columna de respuesta '{response_col}' no está en el DataFrame.")

    # Determinar automáticamente las columnas predictoras
    predictor_cols = df.columns.drop(response_col)

    # Crear DataFrame para características de retardo
    lag_features = pd.DataFrame(index=df.index)

    for col in predictor_cols:
        for lag in range(1, num_lags + 1):
            lag_features[f'{col}_lag_{lag}'] = df[col].shift(lag)

    # Construir DataFrame final
    final_df = lag_features.copy()

    # Agregar variable de respuesta desplazada
    if response_shift != 0:
        final_df[response_col] = df[response_col].shift(-response_shift + 1)
    else:
        final_df[response_col] = df[response_col]

    # Eliminar valores faltantes y resetear índice
    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    # Exportar si se especifica una ruta
    if output_path:
        try:
            directory = os.path.dirname(output_path)
            if directory:
                os.makedirs(directory, exist_ok=True)
            final_df.to_excel(output_path, index=False)
            print(f"El DataFrame con características de retardo ha sido exportado exitosamente a: {output_path}")
        except Exception as e:
            print(f"Error al exportar el DataFrame a Excel: {e}")

    return final_df


In [ ]:
def sliding_window_regression_models_lagged(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[7, 14, 21],
    test_window=1,
    modelos=[
        {'name': 'Kneighboors', 'ModelClass': KNeighborsRegressor, 'params': {'n_neighbors': [5, 10,15], 'algorithm': ['ball_tree', 'kd_tree'], 'leaf_size':[30,60], 'n_jobs':[-1], 'weights': ['uniform', 'distance']}},
        {'name': 'SVR', 'ModelClass': SVR, 'params': {
          'C': [0.1, 1.0, 10],            # Regularización (más alto = menos tolerancia al error)
          'epsilon': [0.01, 0.1, 0.2],     # Margen de tolerancia al error sin penalización
          'kernel': ['rbf','linear'],          # Función de kernel (RBF = radial basis function, muy usada)
        }
      }
    ],
    save_path='./progreso'
):
    os.makedirs(save_path, exist_ok=True)
    resultados_por_T = {}

    for T in tqdm(T_values, desc="Procesando T", unit="ventana", leave=True):
        print(f"\n🧪 Ventana T = {T} días")
        T_hours = T * 24
        test_hours = test_window * 24
        total_windows = len(df) - (2 * T_hours) - test_hours + 1
        print(f'😎 Total windows {total_windows}')

        output_path = os.path.join(save_path, f's_resultados_T{T}.csv')
        split_dir = os.path.join(save_path, f'splits_T{T}')
        os.makedirs(split_dir, exist_ok=True)

        if os.path.exists(output_path):
            df_prev = pd.read_csv(output_path)
            if 'params' in df_prev:
                df_prev['params'] = df_prev['params'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else (None if pd.isna(x) else x)
                )
        else:
            df_prev = pd.DataFrame(columns=[
                'modelo', 'params', 'T_dias', 'T_horas', 'MAPE', 'MAE', 'RMSE', 'MSE', 'R2', 'LjungBox_p'
            ])

        # ⚡ Generar o cargar splits desde .npz
        splits = []
        for i, start in enumerate(tqdm(range(0, total_windows, 24), desc="Generando splits", leave=True)):
            split_file = os.path.join(split_dir, f'split_{i}.npz')

            if os.path.exists(split_file):
                data = np.load(split_file, allow_pickle=True)
                X_train, y_train, X_test, y_test = data['X_train'], data['y_train'], data['X_test'], data['y_test']
            else:
                data_window = df.iloc[start: start + (2 * T_hours) + test_hours].copy()

                temp_df = create_shifted_lagged_features(
                    data_window,
                    response_col=target_col,
                    num_lags=T_hours,
                    response_shift=1
                )

                train = temp_df.iloc[:-test_hours]
                test = temp_df.iloc[-test_hours:]

                X_train = train.drop(columns=[target_col]).values
                y_train = train[target_col].values
                X_test = test.drop(columns=[target_col]).values
                y_test = test[target_col].values

                np.savez_compressed(split_file,
                                    X_train=X_train,
                                    y_train=y_train,
                                    X_test=X_test,
                                    y_test=y_test)

            splits.append((X_train, y_train, X_test, y_test))

        # 🧠 Crear modelos
        models = []
        for modelo in tqdm(modelos, desc="Construyendo modelos", leave=False):
            model_name = modelo['name']
            ModelClass = modelo['ModelClass']
            param_grid = modelo['params']

            if param_grid is None:
                models.append({'name': model_name, 'model': ModelClass(), 'params': True})
            else:
                for combo in product(*param_grid.values()):
                    param_dict = dict(zip(param_grid.keys(), combo))
                    model_instance = ModelClass(**param_dict)
                    models.append({
                        'name': model_name,
                        'model': model_instance,
                        'params': param_dict
                    })

        # 🔍 Evaluar modelos
        for m in tqdm(models, desc="Evaluando modelos", leave=True):
            ya_evaluado = ((df_prev['modelo'] == m['name']) &
                           (df_prev['params'].apply(lambda p: p == m['params']))).any()

            if not df_prev.empty and ya_evaluado:
                print(f"✓ Modelo {m['name']} con {m['params']} ya evaluado para T={T}. Saltando...")
                continue

            resultados = {
                'modelo': m['name'],
                'params': m['params'],
                'T_dias': T,
                'T_horas': T_hours,
                'MAPE': [],
                'MAE': [],
                'RMSE': [],
                'MSE': [],
                'R2': [],
                'LjungBox_p': []
            }

            for X_train, y_train, X_test, y_test in tqdm(splits, desc=f"{m['name']} (T={T}d)", leave=True):
                model = m['model']
                #Escalo variables
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.fit_transform(X_test)
                
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                residuals = y_test - y_pred

                resultados['MAPE'].append(mean_absolute_percentage_error(y_test, y_pred))
                resultados['MAE'].append(mean_absolute_error(y_test, y_pred))
                resultados['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
                resultados['MSE'].append(mean_squared_error(y_test, y_pred))
                resultados['R2'].append(r2_score(y_test, y_pred))

                if len(residuals) >= 2:
                    ljung_p = acorr_ljungbox(residuals, lags=[1], return_df=True)['lb_pvalue'].iloc[0]
                else:
                    ljung_p = np.nan

                resultados['LjungBox_p'].append(ljung_p)

            nuevo_row = pd.DataFrame([{
                'modelo': resultados['modelo'],
                'params': resultados['params'],
                'T_dias': resultados['T_dias'],
                'T_horas': resultados['T_horas'],
                'MAPE': np.mean(resultados['MAPE']),
                'MAE': np.mean(resultados['MAE']),
                'RMSE': np.mean(resultados['RMSE']),
                'MSE': np.mean(resultados['MSE']),
                'R2': np.mean(resultados['R2']),
                'LjungBox_p': np.nanmean(resultados['LjungBox_p'])
            }])

            df_prev = pd.concat([df_prev, nuevo_row], ignore_index=True)
            df_prev.to_csv(output_path, index=False)
            print(f"📦 Guardado modelo {m['name']} con {m['params']} en T={T}")

        resultados_por_T[T] = df_prev
        print(f"✔ Resultados finales guardados en: {output_path}")

    return resultados_por_T


In [ ]:
results = sliding_window_regression_models_lagged(
    df,
    target_col='WIND_VEL_HOR',
    T_values=[21],
    test_window=1,
    modelos=[
        {'name': 'Kneighboors', 'ModelClass': KNeighborsRegressor, 'params': {'n_neighbors': [5, 10,15], 'algorithm': ['ball_tree', 'kd_tree'], 'leaf_size':[30,60], 'n_jobs':[-1], 'weights': ['uniform', 'distance']}},
        {'name': 'SVR', 'ModelClass': SVR, 'params': {
          'C': [0.1, 1.0, 10],            # Regularización (más alto = menos tolerancia al error)
          'epsilon': [0.01, 0.1, 0.2],     # Margen de tolerancia al error sin penalización
          'kernel': ['rbf','linear'],          # Función de kernel (RBF = radial basis function, muy usada)
        }
      }
    ],
    save_path='./progreso'
)

Procesando T:   0%|          | 0/1 [00:00<?, ?ventana/s]


🧪 Ventana T = 21 días
😎 Total windows 86662


Generando splits:   0%|          | 0/3611 [00:00<?, ?it/s]